In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, NMF

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss


from pprint import pprint

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

os.getcwd()

'C:\\Users\\josep'

In [4]:
os.chdir("C:\\Users\\josep\\ud120-projects\\final_project")

In [5]:
os.listdir()

['.ipynb_checkpoints',
 'emails_by_address',
 'enron61702insiderpay.pdf',
 'final_project_dataset.pkl',
 'final_project_dataset.pkl_unix.pkl',
 'final_project_dataset_modified.pkl',
 'my_classifier.pkl',
 'my_dataset.pkl',
 'my_feature_list.pkl',
 'poi_email_addresses.py',
 'poi_id.py',
 'poi_names.txt',
 'result_df.pkl',
 'tester.py']



*Had to implement the following code to get the project data to load properly:* 

Source info: https://stackoverflow.com/questions/45368255/error-in-loading-pickle


In [6]:
original = "final_project_dataset.pkl"
destination = "final_project_dataset.pkl_unix.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))

print("Done. Saved %s bytes." % (len(content)-outsize))

Done. Saved 6705 bytes.


#### Opened source file:

In [7]:
with open("final_project_dataset.pkl_unix.pkl", "rb") as data_file:
    data_dict = pkl.load(data_file)

# make a pandas dataframe to explore

enron_df = pd.DataFrame.from_dict(data_dict,orient='index')

In [8]:
# check the data structure

enron_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   salary                     146 non-null    object
 1   to_messages                146 non-null    object
 2   deferral_payments          146 non-null    object
 3   total_payments             146 non-null    object
 4   loan_advances              146 non-null    object
 5   bonus                      146 non-null    object
 6   email_address              146 non-null    object
 7   restricted_stock_deferred  146 non-null    object
 8   deferred_income            146 non-null    object
 9   total_stock_value          146 non-null    object
 10  expenses                   146 non-null    object
 11  from_poi_to_this_person    146 non-null    object
 12  exercised_stock_options    146 non-null    object
 13  from_messages              146 non-null    object

146 Total entries.  21 total features (columns)

In [9]:
# Checking the dataset

enron_df.head()

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
METTS MARK,365788,807,NaN,1061827,NaN,600000,mark.metts@enron.com,NaN,NaN,585062,...,38,NaN,29,1740,1,False,NaN,702,585062,NaN
BAXTER JOHN C,267102,NaN,1295738,5634343,NaN,1200000,NaN,NaN,-1386055,10623258,...,NaN,6680544,NaN,2660303,NaN,False,1586055,NaN,3942714,NaN
ELLIOTT STEVEN,170941,NaN,NaN,211725,NaN,350000,steven.elliott@enron.com,NaN,-400729,6678735,...,NaN,4890344,NaN,12961,NaN,False,NaN,NaN,1788391,NaN
CORDES WILLIAM R,NaN,764,NaN,NaN,NaN,NaN,bill.cordes@enron.com,NaN,NaN,1038185,...,10,651850,12,NaN,0,False,NaN,58,386335,NaN
HANNON KEVIN P,243293,1045,NaN,288682,NaN,1500000,kevin.hannon@enron.com,NaN,-3117011,6391065,...,32,5538001,32,11350,21,True,1617011,1035,853064,NaN


In [10]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Convert dictionary to DataFrame

enron_df = pd.DataFrame.from_dict(data_dict, orient = 'index', dtype = float)

# reorganize columns

enron_df = enron_df[
['salary',
'bonus',
'long_term_incentive',
'deferred_income',
'deferral_payments',
'loan_advances',
'other',
'expenses',
'director_fees',
'total_payments',
 'exercised_stock_options',
'restricted_stock',
 'restricted_stock_deferred',
 'total_stock_value',
 'email_address',
 'to_messages',
 'shared_receipt_with_poi',
 'from_messages',
 'from_this_person_to_poi',
 'poi',
 'from_poi_to_this_person']]

In [11]:
# Check the dataset

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

enron_df

,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,email_address,to_messages,shared_receipt_with_poi,from_messages,from_this_person_to_poi,poi,from_poi_to_this_person
METTS MARK,365788.000,600000.000,NaN,NaN,NaN,NaN,1740.000,94299.000,NaN,1061827.000,NaN,585062.000,NaN,585062.000,mark.metts@enron.com,807.000,702.000,29.000,1.000,0.000,38.000
BAXTER JOHN C,267102.000,1200000.000,1586055.000,-1386055.000,1295738.000,NaN,2660303.000,11200.000,NaN,5634343.000,6680544.000,3942714.000,NaN,10623258.000,NaN,NaN,NaN,NaN,NaN,0.000,NaN
ELLIOTT STEVEN,170941.000,350000.000,NaN,-400729.000,NaN,NaN,12961.000,78552.000,NaN,211725.000,4890344.000,1788391.000,NaN,6678735.000,steven.elliott@enron.com,NaN,NaN,NaN,NaN,0.000,NaN
CORDES WILLIAM R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,651850.000,386335.000,NaN,1038185.000,bill.cordes@enron.com,764.000,58.000,12.000,0.000,0.000,10.000
HANNON KEVIN P,243293.000,1500000.000,1617011.000,-3117011.000,NaN,NaN,11350.000,34039.000,NaN,288682.000,5538001.000,853064.000,NaN,6391065.000,kevin.hannon@enron.com,1045.000,1035.000,32.000,21.000,1.000,32.000
MORDAUNT KRISTINA M,267093.000,325000.000,NaN,NaN,NaN,NaN,1411.000,35018.000,NaN,628522.000,NaN,208510.000,NaN,208510.000,kristina.mordaunt@enron.com,NaN,NaN,NaN,NaN,0.000,NaN
MEYER ROCKFORD G,NaN,NaN,NaN,NaN,1848227.000,NaN,NaN,NaN,NaN,1848227.000,493489.000,462384.000,NaN,955873.000,rockford.meyer@enron.com,232.000,22.000,28.000,0.000,0.000,0.000
MCMAHON JEFFREY,370448.000,2600000.000,694862.000,NaN,NaN,NaN,297353.000,137108.000,NaN,4099771.000,1104054.000,558801.000,NaN,1662855.000,jeffrey.mcmahon@enron.com,2355.000,2228.000,48.000,26.000,0.000,58.000
HAEDICKE MARK E,374125.000,1150000.000,983346.000,-934484.000,2157527.000,NaN,52382.000,76169.000,NaN,3859065.000,608750.000,524169.000,-329825.000,803094.000,mark.haedicke@enron.com,4009.000,1847.000,1941.000,61.000,0.000,180.000
PIPER GREGORY F,197091.000,400000.000,NaN,-33333.000,1130036.000,NaN,778.000,43057.000,NaN,1737629.000,880290.000,409554.000,-409554.000,880290.000,greg.piper@enron.com,1238.000,742.000,222.000,48.000,0.000,61.000


In [12]:
names = sorted(data_dict.keys())  #sort names of Enron employees in dataset by first letter of last name

print('Sorted list of Enron employees by last name:\n')
pprint(names)

Sorted list of Enron employees by last name:

['ALLEN PHILLIP K',
 'BADUM JAMES P',
 'BANNANTINE JAMES M',
 'BAXTER JOHN C',
 'BAY FRANKLIN R',
 'BAZELIDES PHILIP J',
 'BECK SALLY W',
 'BELDEN TIMOTHY N',
 'BELFER ROBERT',
 'BERBERIAN DAVID',
 'BERGSIEKER RICHARD P',
 'BHATNAGAR SANJAY',
 'BIBI PHILIPPE A',
 'BLACHMAN JEREMY M',
 'BLAKE JR. NORMAN P',
 'BOWEN JR RAYMOND M',
 'BROWN MICHAEL',
 'BUCHANAN HAROLD G',
 'BUTTS ROBERT H',
 'BUY RICHARD B',
 'CALGER CHRISTOPHER F',
 'CARTER REBECCA C',
 'CAUSEY RICHARD A',
 'CHAN RONNIE',
 'CHRISTODOULOU DIOMEDES',
 'CLINE KENNETH W',
 'COLWELL WESLEY',
 'CORDES WILLIAM R',
 'COX DAVID',
 'CUMBERLAND MICHAEL S',
 'DEFFNER JOSEPH M',
 'DELAINEY DAVID W',
 'DERRICK JR. JAMES V',
 'DETMERING TIMOTHY J',
 'DIETRICH JANET R',
 'DIMICHELE RICHARD G',
 'DODSON KEITH',
 'DONAHUE JR JEFFREY M',
 'DUNCAN JOHN H',
 'DURAN WILLIAM D',
 'ECHOLS JOHN B',
 'ELLIOTT STEVEN',
 'FALLON JAMES B',
 'FASTOW ANDREW S',
 'FITZGERALD JAY L',
 'FOWLER PEGGY',
 'FOY JO

## Outlier Investigation and Removal

Study of last two queries reveals two entries of concern:  "The Travel Agency in the Park" and "Total".  

Will remove as they are obviously not individuals and a majority of the features have 'Nan' as a value.

In [13]:
data_dict.pop('TOTAL',0)
data_dict.pop('THE TRAVEL AGENCY IN THE PARK',0)

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 'NaN',
 'total_payments': 362096,
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'deferred_income': 'NaN',
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 'NaN',
 'from_messages': 'NaN',
 'other': 362096,
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'restricted_stock': 'NaN',
 'director_fees': 'NaN'}

Next, I want to search each individual datapoint to see if any records exist with similiar concerns as the 2 I removed above.  I will start by searching the feature "Total Payments" to isolate individuals with no entry. 

In [14]:
# List of people with no total payment data

for entry in data_dict:
    if data_dict[entry]['total_payments'] == 'NaN':
        print(entry)

CORDES WILLIAM R
LOWRY CHARLES P
CHAN RONNIE
WHALEY DAVID A
CLINE KENNETH W
LEWIS RICHARD
POWERS WILLIAM
PIRO JIM
WROBEL BRUCE
MCDONALD REBECCA
GATHMANN WILLIAM D
GILLIS JOHN
MORAN MICHAEL P
LOCKHART EUGENE E
SHERRICK JEFFREY B
FOWLER PEGGY
CHRISTODOULOU DIOMEDES
HUGHES JAMES A
HAYSLETT RODERICK J
MCCARTY DANNY J
SCRIMSHAW MATTHEW


Now, I will run another query using an additional feature to drill down and try to isolate individuals with little to no valuable entries.  'Total Stock Value' seems like an obvious place to start since it is potential compensation that may not have been paid out or included in total payment data. 

In [15]:
# List of people with no total payment data and no stock option data

for entry in data_dict:
    if data_dict[entry]['total_payments'] == 'NaN' and data_dict[entry]['total_stock_value'] == 'NaN':
        print(entry)

CHAN RONNIE
POWERS WILLIAM
LOCKHART EUGENE E


In [16]:
# Investigating CHAN RONNIE

data_dict['CHAN RONNIE']

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 'NaN',
 'total_payments': 'NaN',
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'NaN',
 'restricted_stock_deferred': -32460,
 'deferred_income': -98784,
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 'NaN',
 'from_messages': 'NaN',
 'other': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'restricted_stock': 32460,
 'director_fees': 98784}

#### Most of the entries contain no data.  Entries with data are canceled out by negative values in other columns.  Will remove this datapoint. 

In [17]:
# Investigating POWERS WILLIAM

data_dict['POWERS WILLIAM']

{'salary': 'NaN',
 'to_messages': 653,
 'deferral_payments': 'NaN',
 'total_payments': 'NaN',
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'ken.powers@enron.com',
 'restricted_stock_deferred': 'NaN',
 'deferred_income': -17500,
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 0,
 'exercised_stock_options': 'NaN',
 'from_messages': 26,
 'other': 'NaN',
 'from_this_person_to_poi': 0,
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 12,
 'restricted_stock': 'NaN',
 'director_fees': 17500}

#### This individual datapoint does not contain much financial data.  However, there are several relevant values related to communications.  I will retain this datapoint in future investigations.

In [18]:
# Investigating LOCKHART EUGENE E

data_dict['LOCKHART EUGENE E']

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 'NaN',
 'total_payments': 'NaN',
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'deferred_income': 'NaN',
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 'NaN',
 'from_messages': 'NaN',
 'other': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'restricted_stock': 'NaN',
 'director_fees': 'NaN'}

#### Again, most of the entries contain no data.  Entries with data are canceled out by negative values in other columns.  Will remove this datapoint. 

In [19]:
# Removing the 2 outlier datapoints as outlined above

data_dict.pop('LOCKHART EUGENE E', 0)
data_dict.pop('CHAN RONNIE', 0)

{'salary': 'NaN',
 'to_messages': 'NaN',
 'deferral_payments': 'NaN',
 'total_payments': 'NaN',
 'loan_advances': 'NaN',
 'bonus': 'NaN',
 'email_address': 'NaN',
 'restricted_stock_deferred': -32460,
 'deferred_income': -98784,
 'total_stock_value': 'NaN',
 'expenses': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'exercised_stock_options': 'NaN',
 'from_messages': 'NaN',
 'other': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'poi': False,
 'long_term_incentive': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'restricted_stock': 32460,
 'director_fees': 98784}

In [20]:
names = sorted(data_dict.keys())  #sort names of Enron employees in dataset by first letter of last name

print('Sorted list of Enron employees by last name:\n')
pprint(names)

Sorted list of Enron employees by last name:

['ALLEN PHILLIP K',
 'BADUM JAMES P',
 'BANNANTINE JAMES M',
 'BAXTER JOHN C',
 'BAY FRANKLIN R',
 'BAZELIDES PHILIP J',
 'BECK SALLY W',
 'BELDEN TIMOTHY N',
 'BELFER ROBERT',
 'BERBERIAN DAVID',
 'BERGSIEKER RICHARD P',
 'BHATNAGAR SANJAY',
 'BIBI PHILIPPE A',
 'BLACHMAN JEREMY M',
 'BLAKE JR. NORMAN P',
 'BOWEN JR RAYMOND M',
 'BROWN MICHAEL',
 'BUCHANAN HAROLD G',
 'BUTTS ROBERT H',
 'BUY RICHARD B',
 'CALGER CHRISTOPHER F',
 'CARTER REBECCA C',
 'CAUSEY RICHARD A',
 'CHRISTODOULOU DIOMEDES',
 'CLINE KENNETH W',
 'COLWELL WESLEY',
 'CORDES WILLIAM R',
 'COX DAVID',
 'CUMBERLAND MICHAEL S',
 'DEFFNER JOSEPH M',
 'DELAINEY DAVID W',
 'DERRICK JR. JAMES V',
 'DETMERING TIMOTHY J',
 'DIETRICH JANET R',
 'DIMICHELE RICHARD G',
 'DODSON KEITH',
 'DONAHUE JR JEFFREY M',
 'DUNCAN JOHN H',
 'DURAN WILLIAM D',
 'ECHOLS JOHN B',
 'ELLIOTT STEVEN',
 'FALLON JAMES B',
 'FASTOW ANDREW S',
 'FITZGERALD JAY L',
 'FOWLER PEGGY',
 'FOY JOE',
 'FREVERT MA

#### Verified entries removed.

In [21]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Convert dictionary to DataFrame
enron_df = pd.DataFrame.from_dict(data_dict, orient = 'index', dtype = float)

# reorganize columns
enron_df = enron_df[
['salary',
'bonus',
'long_term_incentive',
'deferred_income',
'deferral_payments',
'loan_advances',
'other',
'expenses',
'director_fees',
'total_payments',
 'exercised_stock_options',
'restricted_stock',
 'restricted_stock_deferred',
 'total_stock_value',
 'email_address',
 'to_messages',
 'shared_receipt_with_poi',
 'from_messages',
 'from_this_person_to_poi',
 'poi',
 'from_poi_to_this_person']]

In [22]:
enron_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   salary                     94 non-null     float64
 1   bonus                      81 non-null     float64
 2   long_term_incentive        65 non-null     float64
 3   deferred_income            47 non-null     float64
 4   deferral_payments          38 non-null     float64
 5   loan_advances              3 non-null      float64
 6   other                      91 non-null     float64
 7   expenses                   94 non-null     float64
 8   director_fees              15 non-null     float64
 9   total_payments             123 non-null    float64
 10  exercised_stock_options    101 non-null    float64
 11  restricted_stock           108 non-null    float64
 12  restricted_stock_deferred  16 non-null     float64
 13  total_stock_value          125 non

In [23]:
### Total Number of Data Points
print('Total Number of data points: %d' %len(data_dict))

Total Number of data points: 142


#### * Verified 21 total Columns -- just like the data structure queries above
#### * Index query reveals 4 concerning entries now removed.

In [24]:
# Saving the new dataset without the outliers

my_dataset = data_dict

## Feature Exploration

Now I can begin to isolate Persons of Interest(POIs) based on the boolean feature "POI".

In [25]:
# Counting the POIs
print("number of poi: {}".format(enron_df[enron_df['poi']==True]['poi'].count()))

# Counting the NON-POIs
print("number of non_poi: {}".format(enron_df[enron_df['poi']==False]['poi'].count()))

number of poi: 18
number of non_poi: 124


In [26]:
# print the POIs

index = enron_df.index

condition_poi = enron_df["poi"] == True


person_poi = index[condition_poi]

#arrange in alphabetical order

person_poi_list = sorted(person_poi.tolist())

print("The following are individuals labeled as POI (Person of Interest):\n")
print('\n'.join(map(str, person_poi_list)))

The following are individuals labeled as POI (Person of Interest):

BELDEN TIMOTHY N
BOWEN JR RAYMOND M
CALGER CHRISTOPHER F
CAUSEY RICHARD A
COLWELL WESLEY
DELAINEY DAVID W
FASTOW ANDREW S
GLISAN JR BEN F
HANNON KEVIN P
HIRKO JOSEPH
KOENIG MARK E
KOPPER MICHAEL J
LAY KENNETH L
RICE KENNETH D
RIEKER PAULA H
SHELBY REX
SKILLING JEFFREY K
YEAGER F SCOTT


Query of random record to validate features and entries:

In [27]:
print('Example Value Dictionary of Features:\n')
pprint(data_dict['METTS MARK']) 
pprint(len(data_dict['METTS MARK']))

Example Value Dictionary of Features:

{'bonus': 600000,
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'mark.metts@enron.com',
 'exercised_stock_options': 'NaN',
 'expenses': 94299,
 'from_messages': 29,
 'from_poi_to_this_person': 38,
 'from_this_person_to_poi': 1,
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 1740,
 'poi': False,
 'restricted_stock': 585062,
 'restricted_stock_deferred': 'NaN',
 'salary': 365788,
 'shared_receipt_with_poi': 702,
 'to_messages': 807,
 'total_payments': 1061827,
 'total_stock_value': 585062}
21


In [28]:
# List of original features

features_list = [
 'poi',
 'bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'email_address',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'long_term_incentive',
 'other',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value'
]

In [29]:
# Count number of entries missing values for each feature

features_nan = {}

for name, feature in data_dict.items():
    for feature_key, val in feature.items():
        if val == 'NaN':
            # Assign 0 to value
            feature[feature_key] = 0
            if features_nan.get(feature_key):
                features_nan[feature_key] = features_nan.get(feature_key) + 1
            else:
                features_nan[feature_key] = 1

print('# of Missing Values by Feature:\n')
print("{:<25} {:<5}".format('FEATURE', 'COUNT'))
print('-------------------------------')
for key, value in features_nan.items(): 
    print("{:<25} {:<5}".format(key, value))

# of Missing Values by Feature:

FEATURE                   COUNT
-------------------------------
deferral_payments         104  
loan_advances             139  
restricted_stock_deferred 126  
deferred_income           95   
exercised_stock_options   41   
long_term_incentive       77   
director_fees             127  
to_messages               56   
email_address             31   
from_poi_to_this_person   56   
from_messages             56   
from_this_person_to_poi   56   
shared_receipt_with_poi   56   
salary                    48   
total_payments            19   
bonus                     61   
expenses                  48   
other                     51   
restricted_stock          34   
total_stock_value         17   


#### It is tempting to remove some features here.  However, since POIs represent only 13% of the total number of datapoints, I will include all of these features just in case. 

In [30]:
dataPoints = float(len(my_dataset))  ## count of data points
featureCount = float(len(features_list))  ## count of all features

## calculation of poi's and non-poi's

pois = 0.
nonPois = 0.
for i in my_dataset.values():
    if i["poi"] == 1:
        pois += 1.
    else:
        nonPois += 1.

## data exploration

print("Number of Data Points : ", dataPoints)
print("Number of Poi's : ", pois)
print("Number of non-Poi's : ", nonPois)
print("Number of missing Poi values : ", dataPoints - (pois + nonPois))
print("Poi percentage : %", round((pois / (nonPois + pois)), 2) * 100)  ## poi's percentage for all records
print("Number of Features : ", featureCount)



Number of Data Points :  142.0
Number of Poi's :  18.0
Number of non-Poi's :  124.0
Number of missing Poi values :  0.0
Poi percentage : % 13.0
Number of Features :  21.0


#### Verified new dataset contains appropriate number of executive datapoints after outliers removed.  Still retain 21 total features.  POI proves to be a solid feature as no remaining entries are missing data. 

*Side note:  It's quite concerning that 13% of executives are labeled as POIs!*

In [31]:
import numpy as np

def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True, remove_any_zeroes=False, sort_keys = False):
    """ convert dictionary to numpy array of features
        remove_NaN = True will convert "NaN" string to 0.0
        remove_all_zeroes = True will omit any data points for which
            all the features you seek are 0.0
        remove_any_zeroes = True will omit any data points for which
            any of the features you seek are 0.0
        sort_keys = True sorts keys by alphabetical order. Setting the value as
            a string opens the corresponding pickle file with a preset key
            order (this is used for Python 3 compatibility, and sort_keys
            should be left as False for the course mini-projects).
        NOTE: first feature is assumed to be 'poi' and is not checked for
            removal for zero or missing values.
    """


    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print("error: key ", feature, " not present")
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)


def targetFeatureSplit( data ):
    """ 
        given a numpy array like the one returned from
        featureFormat, separate out the first feature
        and put it into its own list (this should be the 
        quantity you want to predict)

        return targets and features as separate lists

        (sklearn can generally handle both lists and numpy arrays as 
        input formats when training/predicting)
    """

    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )

    return target, features



#### Introduction of included code to format features.  I next want to search for outliers and see if any interesting relationships emerge that I can use as the basis for new features.


### Next, I'll do a search for specific outlier entries.

In [32]:
outlier_POIs = dict()

cases = 0
for column_name in list(enron_df):
    if enron_df[column_name].dtype == 'float' and column_name != 'poi':
        cases +=1
        test_data = enron_df[enron_df[column_name]!=0]
        Q1 = test_data[column_name].quantile(0.25)
        Q3 = test_data[column_name].quantile(0.75)
        IQR = Q3 - Q1

        # Filtering Values between Q1-1.5IQR and Q3+1.5IQR
        query_str = '(@Q1 - 2 * @IQR) >= ' + column_name + ' or  (@Q3 + 2 * @IQR) <=' + column_name
        filtered = test_data.query(query_str)

        print(column_name + ": " + str(len(filtered)))
        filtered = filtered.sort_values(column_name)
        print(filtered[column_name].head(10))
        print("")
        
        for person in filtered.index.tolist():
            try:
               outlier_POIs[person] = 1 + outlier_POIs[person]
            except:
               outlier_POIs[person] = 1 

print("Total Cases:" + str(cases))

salary: 7
BANNANTINE JAMES M       477.000
GRAY RODNEY             6615.000
WHALLEY LAWRENCE G    510364.000
PICKERING MARK R      655037.000
FREVERT MARK A       1060932.000
LAY KENNETH L        1072321.000
SKILLING JEFFREY K   1111258.000
Name: salary, dtype: float64

bonus: 8
WHALLEY LAWRENCE G   3000000.000
DELAINEY DAVID W     3000000.000
KITCHEN LOUISE       3100000.000
ALLEN PHILLIP K      4175000.000
BELDEN TIMOTHY N     5249999.000
SKILLING JEFFREY K   5600000.000
LAY KENNETH L        7000000.000
LAVORATO JOHN J      8000000.000
Name: bonus, dtype: float64

long_term_incentive: 4
LAVORATO JOHN J   2035380.000
ECHOLS JOHN B     2234774.000
LAY KENNETH L     3600000.000
MARTIN AMANDA K   5145434.000
Name: long_term_incentive, dtype: float64

deferred_income: 5
RICE KENNETH D     -3504386.000
FREVERT MARK A     -3367011.000
HANNON KEVIN P     -3117011.000
ALLEN PHILLIP K    -3081055.000
BELDEN TIMOTHY N   -2334434.000
Name: deferred_income, dtype: float64

deferral_payments: 4
AL

### Top 10 outlier individuals:

In [33]:
outlier_POIs_df = pd.DataFrame.from_dict(outlier_POIs, orient = 'index', dtype = int)
outlier_POIs_df.columns = ['Count']
outlier_POIs_df = outlier_POIs_df.sort_values('Count', ascending = False)
outlier_POIs_df.head(10)

,Count
FREVERT MARK A,9
LAY KENNETH L,8
LAVORATO JOHN J,7
BELDEN TIMOTHY N,7
SKILLING JEFFREY K,6
KITCHEN LOUISE,5
BAXTER JOHN C,5
ALLEN PHILLIP K,4
RICE KENNETH D,4
KEAN STEVEN J,4


Next, I want to check to see if any powerful correlations exist that can also shed light on potential new features.

In [34]:
# looking for correlations in the data

enron_df.corr(method ='pearson')

,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,to_messages,shared_receipt_with_poi,from_messages,from_this_person_to_poi,poi,from_poi_to_this_person
salary,1.000,0.523,0.484,-0.354,0.456,0.535,0.607,0.145,NaN,0.579,0.607,0.551,-0.593,0.615,0.187,0.285,-0.004,0.021,0.265,0.179
bonus,0.523,1.000,0.625,-0.412,0.294,0.974,0.376,0.034,NaN,0.569,0.507,0.377,0.118,0.509,0.373,0.549,0.053,0.355,0.302,0.562
long_term_incentive,0.484,0.625,1.000,-0.332,0.042,1.000,0.529,-0.101,NaN,0.518,0.500,0.635,-0.187,0.495,0.134,0.179,-0.072,0.084,0.255,0.213
deferred_income,-0.354,-0.412,-0.332,1.000,-0.881,1.000,-0.185,0.101,-0.461,-0.054,-0.308,-0.127,0.136,-0.298,-0.351,-0.395,-0.320,-0.272,-0.261,-0.179
deferral_payments,0.456,0.294,0.042,-0.881,1.000,-1.000,0.386,0.005,NaN,0.067,0.019,0.115,-0.413,0.076,0.310,0.357,0.322,0.207,-0.098,0.424
loan_advances,0.535,0.974,1.000,1.000,-1.000,1.000,1.000,0.659,NaN,0.992,0.960,1.000,NaN,0.962,0.740,0.292,-0.214,0.935,1.000,0.010
other,0.607,0.376,0.529,-0.185,0.386,1.000,1.000,0.063,NaN,0.826,0.536,0.644,0.012,0.627,0.041,0.123,-0.102,-0.110,0.120,0.111
expenses,0.145,0.034,-0.101,0.101,0.005,0.659,0.063,1.000,-0.505,0.110,0.019,0.043,0.041,0.035,0.155,0.223,0.094,0.015,0.060,-0.059
director_fees,NaN,NaN,NaN,-0.461,NaN,NaN,NaN,-0.505,1.000,0.317,0.623,-1.000,0.664,0.997,-1.000,1.000,1.000,1.000,NaN,NaN
total_payments,0.579,0.569,0.518,-0.054,0.067,0.992,0.826,0.110,0.317,1.000,0.592,0.596,0.942,0.668,0.134,0.191,-0.033,0.012,0.230,0.154


I'll start by examining the above chart to find any strong correlations between features to use as the basis for any new features.

An obvious place to start is with the "from_this_person_to_poi" feature, as this will reveal individuals with a lot of communications with POIs.  

* "loan_advances" emerges as the first obvious choice.  The correlation ratio of 0.935 is extremely strong and is one of the highest on the chart.
* There are no other correlations under this feature strong enough for my liking.  I will create a new feature combining "loan_advances" and "from_this_person_to_poi".
* The new feature will be named "loan_advances_feature"



In [35]:
# Creates a new list by adding two lists together

def get_total_list(list1, list2):
    new_list = []
    for i in my_dataset:
        if my_dataset[i][list1] == 'NaN' or my_dataset[i][list2] == 'NaN':
            new_list.append(0.)
        elif my_dataset[i][list1]>=0:
            new_list.append(float(my_dataset[i][list1]) + float(my_dataset[i][list2]))
    return new_list

In [36]:
# Creation of new feature variable

loan_advances_feature = get_total_list('loan_advances', 'from_this_person_to_poi')

In [37]:
len(loan_advances_feature)

142

In [38]:
# Adding this new feature to the dataset

count = 0
for i in my_dataset:
    my_dataset[i]['loan_advances_feature'] = loan_advances_feature[count]
    count += 1

In [39]:
# Printing all features

print(len((my_dataset['SKILLING JEFFREY K'].keys())))
print(my_dataset['SKILLING JEFFREY K'].keys())

22
dict_keys(['salary', 'to_messages', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'email_address', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'from_poi_to_this_person', 'exercised_stock_options', 'from_messages', 'other', 'from_this_person_to_poi', 'poi', 'long_term_incentive', 'shared_receipt_with_poi', 'restricted_stock', 'director_fees', 'loan_advances_feature'])


#### Verified 'loan_advances_feature' now added to list.

Next, I will add the new feature to features_list and remove the "email_address" feature as it has no value for this exercise.  I will also remove the 'other' feature as it is undefined and might skew the data. 

In [40]:
# Adding new feature to features_list and removing "email_address" feature 

features_list = [
 'poi',
 'bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'loan_advances_feature'
 'long_term_incentive',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value'
]

Now, I want to use Decision Tree to see which set of features best suites my needs

In [41]:
# Decision Tree with original set of features

import numpy as np
np.random.seed(42)
from time import time
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# features_list

features_list = ['poi','salary', 'from_poi_to_this_person', 'from_this_person_to_poi', 'to_messages', 
                 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 
                 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 
                 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 
                 'director_fees', 'deferred_income', 'long_term_incentive']

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree

from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.7894736842105263
Precision:  0.2222222222222222
Recall:  0.2857142857142857


In [42]:
# Decision Tree with updated set of features

import numpy as np
np.random.seed(42)
from time import time
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# features_list

features_list = ['poi', 'bonus', 'deferral_payments', 'deferred_income', 'director_fees',
                 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person',
                 'from_this_person_to_poi', 'loan_advances', 'loan_advances_feature', 'long_term_incentive',
                 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
                 'to_messages', 'total_payments', 'total_stock_value'
                ]

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree

from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.7894736842105263
Precision:  0.2222222222222222
Recall:  0.2857142857142857


#### Decision Tree returns the exact same results for both set of features.  Will run again without new feature, 'other', or 'email_address'

In [43]:
# Decision Tree with original set of features minus email address and other

import numpy as np
np.random.seed(42)
from time import time
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# features_list

features_list = ['poi', 'bonus', 'deferral_payments', 'deferred_income', 'director_fees',
                 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person',
                 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive',
                 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
                 'to_messages', 'total_payments', 'total_stock_value'
                ]

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree

from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.8070175438596491
Precision:  0.3
Recall:  0.42857142857142855


#### This proves to be the most accurate set of features.

* All of the performance measurements were improved, but not by much.  I will now run Select K Best to isolate the most important features from the dataset.

In [44]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

def selectFeatures(nParam):
    kBest = SelectKBest(k=nParam)
    kBest.fit_transform(features, labels)
    kResult = zip(kBest.get_support(), kBest.scores_, features_list[1:])
    return list(sorted(kResult, key=lambda x: x[1], reverse=True))


resultsAll = selectFeatures("all")

import pprint

pprint.pprint(resultsAll)

[(True, 24.532722463057976, 'exercised_stock_options'),
 (True, 23.898259813869416, 'total_stock_value'),
 (True, 20.524645181851792, 'bonus'),
 (True, 18.003739993113935, 'salary'),
 (True, 11.353769628808733, 'deferred_income'),
 (True, 9.772103538408254, 'long_term_incentive'),
 (True, 9.081517827190943, 'restricted_stock'),
 (True, 8.671773243105207, 'total_payments'),
 (True, 8.432635423024681, 'shared_receipt_with_poi'),
 (True, 7.1253824688830685, 'loan_advances'),
 (True, 5.954544292197293, 'expenses'),
 (True, 5.14221919450697, 'from_poi_to_this_person'),
 (True, 2.3388361146462624, 'from_this_person_to_poi'),
 (True, 1.9683646966682355, 'director_fees'),
 (True, 1.5942560277180795, 'to_messages'),
 (True, 0.2323680582546254, 'deferral_payments'),
 (True, 0.1753832041587958, 'from_messages'),
 (True, 0.06643110199560048, 'restricted_stock_deferred')]


A brief examination of the above results reveals the top five strongest features:

* 'exercised_stock_options'
* 'total_stock_value'
* 'bonus'
* 'salary'
* 'deferred_income

I will now run Decision Tree using the top 4, 5, and 6 features to check which feature set performs the best.

In [45]:
# features_list top 4
features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 'salary']
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.8076923076923077
Precision:  0.2857142857142857
Recall:  0.2857142857142857


In [46]:
# Top 5 features

features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
                 'salary', 'deferred_income']
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.75
Precision:  0.1
Recall:  0.16666666666666666


In [47]:
# Top 6 features

features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
                 'salary', 'deferred_income','long_term_incentive']
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data into training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.75
Precision:  0.1
Recall:  0.16666666666666666


#### Testing reveals the top 4 features performs the best out of all three.  However, the accuracy is only around 80% and precision and recall are both under 0.3 so I still have some work to do.  Rerunning using the top 4 features with the new feature I created above:

In [48]:
# features_list top 4 with new feature

features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 'loan_advances_feature']
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.7692307692307693
Precision:  0.2727272727272727
Recall:  0.42857142857142855


Testing reveals using the top 4 features with my newly created feature returns the best balance of Precision and Recall of the group.  However, Accuracy has decreased and Precision is now below 0.3.  Going to include the top 10 features to see how that changes things: 

In [49]:
features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
                 'salary', 'deferred_income', 'long_term_incentive','restricted_stock',
                 'total_payments', 'shared_receipt_with_poi','loan_advances']

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.8070175438596491
Precision:  0.3
Recall:  0.42857142857142855


Now this I like.  The accuracy is up over 80% and the Precision and Recall are both at or above 0.3.

In [50]:
features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
                 'salary', 'deferred_income', 'long_term_incentive','restricted_stock',
                 'total_payments', 'shared_receipt_with_poi','loan_advances_feature']

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.7719298245614035
Precision:  0.2
Recall:  0.2857142857142857


Introduction of newly created feature has detrimental impact on performance.  Will not use. 

In [51]:
# establish list of features used

features_list = ['poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
                 'salary', 'deferred_income', 'long_term_incentive','restricted_stock',
                 'total_payments', 'shared_receipt_with_poi','loan_advances']

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

# split data inton training and testing
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.4, random_state = 42)

# choose decision tree
from sklearn.tree import DecisionTreeClassifier
t0 = time()
clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

acc = accuracy_score(labels_test, pred)
print('Accuracy: ' + str(acc))
print('Precision: ', precision_score(labels_test, pred))
print('Recall: ', recall_score(labels_test, pred))

Accuracy: 0.7894736842105263
Precision:  0.2727272727272727
Recall:  0.42857142857142855


## Algorithms

In [52]:
# Use of different scaling options to determine which works best

distributions = [
    ('Data after standard scaling',
        StandardScaler().fit_transform(features)),
    ('Data after min-max scaling',
        MinMaxScaler().fit_transform(features)),
    ('Data after max-abs scaling',
        MaxAbsScaler().fit_transform(features)),
    ('Data after power transformation (Yeo-Johnson)',
     PowerTransformer(method='yeo-johnson').fit_transform(features)),
    ('Data after quantile transformation (gaussian pdf)',
        QuantileTransformer(output_distribution='normal')
        .fit_transform(features)),
    ('Data after quantile transformation (uniform pdf)',
        QuantileTransformer(output_distribution='uniform')
        .fit_transform(features))
]

### Classifiers

In [53]:
# list of classifiers

classifiers = [
    ('SVC', SVC()),
    ('DTC', DecisionTreeClassifier()),
    ('RFC', RandomForestClassifier()),
    ('ADA', AdaBoostClassifier()),
    ('GBC', GradientBoostingClassifier())
]

# param grid for different classifiers
param_grid_list = [('SVC', {'C': [1,10,50,100],
                            'kernel': ['linear', 'poly','rbf']}),
                   ('DTC',{'criterion': ['gini','entropy'], 
                           'splitter': ['best','random'], 
                           'min_samples_split' : [2,5,8, 10],
                           'max_depth': [10,25,50,100,200]}),
                   ('RFC', {'n_estimators': [10,50,100], 
                            'criterion': ['gini','entropy'],
                            'max_depth': [3,5,8],
                            'max_features' : ['log2','sqrt']}),
                   ('ADA', {'n_estimators': [10,50,100], 
                            'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
                            'algorithm' : ['SAMME', 'SAMME.R']}),
                   ('GBC', {'loss': ['deviance'],
                            'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
                            'max_depth': [3,5,8],
                            'max_features' : ['log2','sqrt'],
                            'n_estimators': [10,50,100]})
                  ]

Now, I'll use GridSearchCV to loop 

In [54]:
# loop scale features

result_list = []
for x, features in enumerate(distributions):
    # get scaled features from distribution
    scaled_features = distributions[x][1]
    #divide train and test set
    features_train, features_test, labels_train, labels_test = train_test_split(
        scaled_features, labels, test_size=0.33, random_state=42)
    print('###########################################################')
    print('###################### feature ############################')
    print(features[0])
    # loop classifiers
    for y, classifier in classifiers:
        print('################### classifier ############################')
        print(y, classifier)
        print('###########################################################')
        #find matching parameter grid
        for z, items in param_grid_list:
            if y == z:
                print(y, z, items)
                # classifier using GridSearchCV optimizing for f1 score
                clf = GridSearchCV(classifier, items, scoring='f1_weighted', n_jobs=-2)
                clf.fit(features_train, labels_train)
                print('###########################################################')
                print('parameters:' + str(clf.best_params_))
                print('###########################################################')
                print('accuracy training set: {0:.3g}'.format(clf.score(features_train, labels_train)))             
                print('accuracy test set: {0:.3g}'.format(clf.score(features_test, labels_test)))                    
                print('precision_score : {0:.3g}'.format(precision_score(labels_test,clf.predict(features_test))))
                print('recall_score : {0:.3g}'.format(recall_score(labels_test,clf.predict(features_test))))          
                print('f1-score : {0:.3g}'.format(f1_score(labels_test,clf.predict(features_test))))                
                result_list.append({'scaling' : features[0],
                                    'classifier' : classifier.__class__.__name__,
                                    'best_params' : clf.best_params_,
                                    'accuracy_training_set': clf.score(features_train, labels_train),            
                                    'accuracy_test_set': clf.score(features_test, labels_test),                 
                                    'precision_score': precision_score(labels_test,clf.predict(features_test)),
                                    'recall_score': recall_score(labels_test,clf.predict(features_test)),       
                                    'f1_score' : f1_score(labels_test,clf.predict(features_test))})
                # save intermediate results
                result_df = pd.DataFrame(result_list)
                result_df.to_pickle('result_df.pkl')
                
#list to df and pickle for later use
result_df = pd.DataFrame(result_list)
result_df.to_pickle('result_df.pkl')

###########################################################
###################### feature ############################
Data after standard scaling
################### classifier ############################
SVC SVC()
###########################################################
SVC SVC {'C': [1, 10, 50, 100], 'kernel': ['linear', 'poly', 'rbf']}
###########################################################
parameters:{'C': 50, 'kernel': 'poly'}
###########################################################
accuracy training set: 0.941
accuracy test set: 0.829
precision_score : 0.333
recall_score : 0.167
f1-score : 0.222
################### classifier ############################
DTC DecisionTreeClassifier()
###########################################################
DTC DTC {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_split': [2, 5, 8, 10], 'max_depth': [10, 25, 50, 100, 200]}
###########################################################
parameters:{'criterio

In [55]:
result_df.nlargest(5, 'f1_score')

,scaling,classifier,best_params,accuracy_training_set,accuracy_test_set,precision_score,recall_score,f1_score
27,Data after quantile transformation (uniform pdf),RandomForestClassifier,"{'criterion': 'entropy', 'max_depth': 8, 'max_...",0.989,0.878,0.667,0.333,0.444
2,Data after standard scaling,RandomForestClassifier,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.968,0.845,0.400,0.333,0.364
20,Data after quantile transformation (gaussian pdf),SVC,"{'C': 100, 'kernel': 'poly'}",0.978,0.830,0.333,0.333,0.333
11,Data after max-abs scaling,DecisionTreeClassifier,"{'criterion': 'gini', 'max_depth': 50, 'min_sa...",0.928,0.815,0.286,0.333,0.308
5,Data after min-max scaling,SVC,"{'C': 1, 'kernel': 'poly'}",0.879,0.843,0.500,0.167,0.250


# Validation and Evaluation

After several tests, it appears as though DecisionTree performed the best overall.  Accuracy is up around 85 % and precision and recall are consistently over 0.42.

Several differnt types of scaling were used but there was no consistency as to which performed best.  DecisionTree combined with power transformation and quantile transformation seemed to be the most effective combinations.


# Project Summary

### Project Goal

The goal of this project was to examine data related to executives working at the infamous company Enron.  The dataset contains information on 146 individuals, with each record containing 21 features.  The purpose of the project was to utilize machine learning to determine which set of features could most effectively predict attributes a Person of Interest (POI) might display. 18 POIs have previously been identified

Initial examination of the dataset revealed two troublesome records:

* "The Travel Agency in the Park"
* "Total"

I removed those using the .pop() method, then created a for loop to iterate through the remaining entries to check for records with similar concerns and/or little to no data.  I ended up removing two more individual records.  The final dataset contained 142 entries. 

### Features Used

The feature list I ended up sing included the following:

('poi', 'exercised_stock_options', 'total_stock_value', 'bonus', 
'salary', 'deferred_income','long_term_incentive',
'restricted_stock','total_payments',
'shared_receipt_with_poi', 'loan_advances')

#### Selection Process

I started by removing the 'email_address' and 'other' features, as the information contained in those was irrelevant and/or undefined.  I tried to create a new feature 'loan_advances_feature' that combined the 'loan_advances' and 'from_this_person_to_poi' features.  However, that feature proved no benefit so I ended up not using it in my final feature set. 

I used DecisionTreeClassifier first to evaluate different sets of features.  The feature list with the 2 features removed (as outlined above) proved to be the most accurate, although not quite to my liking.  I then utilized Several scaling options (Standard, MinMax, MaxAbs, PowerTranformer, and QuantileTransformer (Gaussian and Uniform)) along with Select K Best to isolate the most important features fromt the dataset.  I utilized the results from that query to use the top five features using a feature score of 10.0+ as criteria.  

Next, I used DecisionTreeClassifier to test feature sets using the top 4, 5, and 6 top features, along with my newly created feature to see which perfomed the best.  I wasn't happy with the results right away, so I ended up settling on using the top 10 features in my final query. 

### Algorithms

I applied the following algorithms during testing:

* SVC
* DecisionTreeClassifier
* RandomForest
* AdaBoost
* GradientBoosting

Several test runs revealed DecisionTree as the top performer of the group, with GradientBoosting and SVC coming in next.  DecisionTree consistently returned accuracy of 90%+ on training sets and 85%+ on test sets.  Precision and Recall both consisently came in at roughly 0.429 and 0.5, respectively.

AdaBoost did not show up in my top 5 performers during all testing.

### Tuning

The definition of tuning, according to [https://en.wikipedia.org/wiki/Hyperparameter_optimization#:~:text=In%20machine%20learning%2C%20hyperparameter%20optimization,typically%20node%20weights)%20are%20learned.]:

In machine learning, hyperparameter optimization or tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm. A hyperparameter is a parameter whose value is used to control the learning process. By contrast, the values of other parameters (typically node weights) are learned.

The same kind of machine learning model can require different constraints, weights or learning rates to generalize different data patterns. These measures are called hyperparameters, and have to be tuned so that the model can optimally solve the machine learning problem. Hyperparameter optimization finds a tuple of hyperparameters that yields an optimal model which minimizes a predefined loss function on given independent data. The objective function
takes a tuple of hyperparameters and returns the associated loss.  Cross-validation is often used to estimate this generalization performance.


When tuning is not done well, it leads to ineffecient, inconsistent, and irrelevant results.  To validate my findings, I created a for loop to test different combination of classifiers with recommended parameters for each, then looped through scale features to look for the top 5 performing parameters. 

### Validation

A validation dataset is a sample of data held back from training your model that is used to give an estimate of model skill while tuning model’s hyperparameters.

When validation is done wrong, it can lead to innacurate findings.  It's important to find desirable symmetry between training and test sets. 

To validate my findings, I created a for loop to test different combinations of classifiers and scaling to isolate the best performers.  I then ran several test runs and took note of the top 5 results and looked for consistency. 

### Testing

Testing revealed accuracy in the range of 85%. 

Precision and Recall came in at 0.429 and 0.5, respectively.  

In [56]:
import pickle
import sys
from sklearn.model_selection import StratifiedShuffleSplit
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit

PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\
\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

In [57]:
def test_classifier(clf, dataset, feature_list, folds = 1000):
    data = featureFormat(dataset, feature_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    cv = StratifiedShuffleSplit(n_splits=folds, random_state=42)
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv.split(features, labels): 
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print("Warning: Found a predicted label not == 0 or 1.")
                print("All predictions should take value 0 or 1.")
                print("Evaluating performance for processed predictions:")
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print(clf)
        print(PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5))
        print(RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives))
        print("")
    except:
        print("Got a divide by zero when trying out:", clf)
        print("Precision or recall may be undefined due to a lack of true positive predicitons.")

In [58]:
def dump_classifier_and_data(clf, dataset, feature_list):
    with open(CLF_PICKLE_FILENAME, "wb") as clf_outfile:
        pickle.dump(clf, clf_outfile)
    with open(DATASET_PICKLE_FILENAME, "wb") as dataset_outfile:
        pickle.dump(dataset, dataset_outfile)
    with open(FEATURE_LIST_FILENAME, "wb") as featurelist_outfile:
        pickle.dump(feature_list, featurelist_outfile)

In [59]:
CLF_PICKLE_FILENAME = "my_classifier.pkl"
DATASET_PICKLE_FILENAME = "my_dataset.pkl"
FEATURE_LIST_FILENAME = "my_feature_list.pkl"

In [60]:
dump_classifier_and_data(clf, my_dataset, features_list)

In [61]:
def load_classifier_and_data():
    with open(CLF_PICKLE_FILENAME, "rb") as clf_infile:
        clf = pickle.load(clf_infile)
    with open(DATASET_PICKLE_FILENAME, "rb") as dataset_infile:
        dataset = pickle.load(dataset_infile)
    with open(FEATURE_LIST_FILENAME, "rb") as featurelist_infile:
        feature_list = pickle.load(featurelist_infile)
    return clf, dataset, feature_list

In [62]:
def main():
    ### load up student's classifier, dataset, and feature_list
    clf, dataset, feature_list = load_classifier_and_data()
    ### Run testing script
    test_classifier(clf, dataset, feature_list)